In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import math
from collections import Counter
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/ufcdata/data.csv')

The modern weight classes were introduced at UFC 33 in 28th September 2001. Prior to this, weight classes were very different, lightweight was under 90kg and heavyweight above 90kg. 
Thus I mark every fight before the ufc 33 as pre UFC 33 lighttweight.

In [ ]:
for fight in df.itertuples():
    if fight.date < '2001-06-29':
        df.at[fight.Index,'weight_class'] = 'Pre UFC33 ' + fight.weight_class


Usually the red corner is the betting favourite in fights. I want to see which corner wins more often.

In [ ]:
df.Winner.value_counts()

Given that red does win more fights, What percent of fights of is won by red corner?

In [ ]:
'The red corner wins '+str(round(df['Winner'].value_counts()['Red']/df['Winner'].count(),4)*100)+'% of fights'

The data we have seperates fighter into Red corner and Blue Corner, however it would be more interesting to seperate fighters by winner and loser.
the following code creates winner stat columns and loser columns and copys over stats from red or blue depending on which corner won or lost.

In [ ]:
#create columns called winner and loser
stats = list()
red_stats = list()
blue_stats = list()

for column_name in list(df):
    if column_name[0:2] == 'R_':
        red_stats.append(column_name) #add to red list
        stats.append(column_name[1:]) # _KD to list of stats to make winner stats later
    elif column_name[0:2] == 'B_':
        blue_stats.append(column_name) #add to red list
        
winner_stats = ['winner'+stat for stat in stats]
loser_stats = ['loser'+stat for stat in stats]

for col in winner_stats:
    df[col]= None
    
for col in loser_stats:
    df[col] = None
    
for fight in df.itertuples():
    if fight.Winner == 'Red':
        #print(df.loc[fight.Index, blue_stats])
        df.loc[fight.Index,winner_stats] = list(df.loc[fight.Index,red_stats])
        df.loc[fight.Index,loser_stats] = list(df.loc[fight.Index,blue_stats])
    elif fight.Winner == 'Blue':
        df.loc[fight.Index,winner_stats] = list(df.loc[fight.Index,blue_stats])
        df.loc[fight.Index,loser_stats] = list(df.loc[fight.Index,red_stats])

drop the red and blue columns as we have winner and loser now

In [ ]:
 df=df.drop(columns = red_stats)
 df=df.drop(columns = blue_stats)


Find the mean of each column to check if I can see a significant difference between winner and loser by eye.

In [ ]:
overall_mean = df.mean(axis=0).to_dict()

for figure in overall_mean.items():
    print(figure)

I want to try see if the average winner has greater career stats than the average loser
The following code places a 1 in a column is the winner had a higher career stat than the corresponding loser stat.
For example, if winner's career average takedown is greater than the loser average takedown, then the column gets a 1.

In [ ]:
#winner minus loser, if postive put a 1, else a 0
for fight in df.itertuples():
    for stat in stats:
        try: #because some stats will return strings such as names
            if (df.at[fight.Index,'winner'+stat]) - (df.at[fight.Index,'loser'+stat]) > 0:
                df.at[fight.Index,'isGreater'+stat] = 1
            else:
                df.at[fight.Index,'isGreater'+stat] = 0
        except:
            continue

for stat in stats:
    try:
        print(str(stat)+':    '+str(np.mean(df['isGreater'+stat]))) #shows the percentage of times the winner was predicted by greater career stata
    except:
            pass
            

Unfortunatly there is no clear relation between a fighters career stats and how likely that fighter is to win the fight. 
_total_rounds_fought predicted the winner 47.22% of the time. 
Which may beat a bookmakers odds, but doesn't clearly predict the winner better than random chance.


It might be more useful to find if a winner has significantly higher career stat than the loser, surely that would predict sucess.
The following code creates a standarised score, it finds how many standard deviations the winner score is away from the mean and compares that with the how many standard deviations the loser is from the mean,
for example: if Fighter A has a career average takedown 2 standard deviations above the mean. Fighter B has a career takedown 0.4 standard deviation **below** the mean then the score would be calculated as 2 -(-0.4) = 2.4.
We put a 1 in the column to see if the difference between two fighters stats is greater than 1 standard deviation and the fighter with the larger value won the fight.


In [ ]:


#standardise with a z stat: x-y/std(x)

standard_dev = {}

for stat in stats:
    try:
        standard_dev[stat] = np.std(df['loser'+stat])
    except:
        pass

    
#find what the average standardised difference is between winner career stats and loser careeer stats
for fight in df.itertuples():
    for stat in stats:
        try: #because some stats will return strings such as names
            if (abs(df.at[fight.Index,'winner'+stat]) - (df.at[fight.Index,'loser'+stat])) > 1 * standard_dev[stat]:
                df.at[fight.Index,'isGreater1std'+stat] = 1
            else:
                df.at[fight.Index,'isGreater1std'+stat] = 0
            
        except:
            continue

            
for stat in stats:
    try:
        pass
        print(str(stat)+':    '+str(np.mean(df['isGreater1std'+stat]))) #shows the number of times the fighter with career stats greater than 1 standard deviation from their oppenant won the fight
    except:
            pass
            

 Again, there no clear relationship between how many standard deviations away the winner is from the loser.
 Even when experimenting with differenct values, no value shows any real predictive power. 
 It appears career stats may just be very poor predictors of a fighter next fight

It is curious that a fighters average career stats would not predict the winner of a fight, perhaps a look at distribution of the differnce in each category.

In [ ]:
differenced_df = pd.DataFrame()
for fight in df.itertuples():
    for stat in stats:
        try: #because some stats will return strings such as names
            differenced_df.at[fight.Index,'difference_in'+stat] = df.at[fight.Index,'winner'+stat] - df.at[fight.Index,'loser'+stat]
        except:
            continue



In [ ]:
%matplotlib inline
for stat in stats[1:]:
    if stat == '_draw':
        continue
    plt.title(stat)
    frequency_count = round(differenced_df['difference_in'+stat],2).value_counts().to_dict()
    plt.bar(frequency_count.keys(),frequency_count.values())
    plt.show()


from the graphs we can see that the difference of career stats is relatively normally distributed about a mean of approximately 0. Unfortunately this means there is no way to predict the winnner by looking at career stats alone because fihgters career statistics are often too similar to really be able to see any difference in them.